# Pickling Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [ ]:
from cuml.datasets import make_blobs

# Generate synthetic dataset for clustering
X, y = make_blobs(
    n_samples=50,
    n_features=10,
    centers=5,
    cluster_std=0.4,
    random_state=0
)

In [ ]:
from cuml.cluster import KMeans as cuKMeans

# Initialize and fit KMeans model
kmeans_model = cuKMeans(n_clusters=5)
kmeans_fitted = kmeans_model.fit(X)

We recommend to use Pickle protocol 5 as it is more efficient especially for large arrays (models).

In [ ]:
import pickle

# Save the fitted model to disk
pickle.dump(kmeans_fitted, open("kmeans_model.pkl", "wb"), protocol=5)

In [ ]:
# Load the model from disk
kmeans_loaded = pickle.load(open("kmeans_model.pkl", "rb"))

In [ ]:
# Display the loaded model's cluster centers
kmeans_loaded.cluster_centers_

## Using joblib for Model Serialization

joblib is a popular alternative to pickle for serializing machine learning models, offering better performance and compression for large NumPy arrays. It's particularly well-suited for cuML models with many parameters or large datasets. joblib provides efficient memory mapping and faster serialization compared to pickle for ML workloads.


Note that pickle and joblib are often directly compatible, but we do not recommend to rely on that.

In [ ]:
import joblib

kmeans_loaded = joblib.load("kmeans_model.pkl")
kmeans_loaded.cluster_centers_

## Distributed Model Pickling

When working with distributed cuML models using Dask, the distributed estimator wrappers in `cuml.dask` are not designed to be pickled directly. Instead, cuML provides a specialized workflow for persisting distributed models:

1. **Extract the combined model**: After training a distributed model, use the `get_combined_model()` method to extract a single-GPU version of the trained model
2. **Serialize the combined model**: The extracted single-GPU model can be pickled or saved using pickle or joblib, just like any other cuML model
3. **Flexible inference**: The saved model can be used for inference in multiple ways:
   - **Single-GPU inference**: Load the model directly for single-GPU predictions
   - **Distributed inference**: Use the `ParallelPostFit` wrapper from [Dask-ML](https://ml.dask.org/meta-estimators.html) to distribute inference across a Dask cluster

This approach allows you to choose the right amount of resources for both training and inference.

In [ ]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

# Set up Dask cluster
cluster = LocalCUDACluster()
client = Client(cluster)
client

In [ ]:
from cuml.dask.datasets import make_blobs

# Get number of workers
n_workers = len(client.scheduler_info()["workers"].keys())

# Generate distributed dataset
X, y = make_blobs(
    n_samples=5000,
    n_features=30,
    centers=5,
    cluster_std=0.4,
    random_state=0,
    n_parts=n_workers * 5
)

# Persist data in memory
X = X.persist()
y = y.persist()

In [ ]:
from cuml.dask.cluster import KMeans as cuDaskKMeans

# Initialize distributed KMeans model
dask_kmeans_model = cuDaskKMeans(n_clusters=5)

In [ ]:
# Fit the distributed model
# dask_kmeans_fitted = dask_kmeans_model.fit(X)

In [ ]:
import pickle

# Extract single-GPU model and save it
# single_gpu_kmeans = dask_kmeans_fitted.get_combined_model()
# pickle.dump(single_gpu_kmeans, open("kmeans_model.pkl", "wb"))

In [ ]:
# Load the single-GPU model
# single_gpu_kmeans_loaded = pickle.load(open("kmeans_model.pkl", "rb"))

In [ ]:
# Display the loaded model's cluster centers
# single_gpu_kmeans_loaded.cluster_centers_

In [ ]:
# Display the loaded model's cluster centers
kmeans_loaded.cluster_centers_

## Exporting cuML Random Forest models for inferencing on machines without GPUs

You can export cuML Random Forest models and run predictions on machines without NVIDIA GPUs using the [Treelite](https://github.com/dmlc/treelite) library. This enables you to train on GPU-accelerated systems and deploy on CPU-only infrastructure.

### Export Process

1. **Convert to Treelite format**: Use `as_treelite()` to transform your cuML Random Forest model
2. **Serialize the model**: Call `.serialize()` to create a portable checkpoint file
3. **Deploy anywhere**: Install Treelite on the target machine and load the model for inference

Treelite provides optimized CPU inference with efficient serialization, making it ideal for production deployment.

In [ ]:
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.datasets import load_iris
import numpy as np

# Load and prepare iris dataset
X, y = load_iris(return_X_y=True)
X, y = X.astype(np.float32), y.astype(np.int32)

# Train Random Forest model
rf_model = cuRF(max_depth=3, random_state=0, n_estimators=10)
rf_fitted = rf_model.fit(X, y)

# Export cuML RF model as Treelite checkpoint
rf_fitted.as_treelite().serialize(checkpoint_path)

2. Copy the generated checkpoint file `checkpoint.tl` to another machine on which you'd like to run predictions.

3. On the target machine, install Treelite by running `pip install treelite` or `conda install -c conda-forge treelite`. The machine does not need to have NVIDIA GPUs and does not need to have cuML installed.

4. You can now load the model from the checkpoint, by running the following on the target machine:

In [ ]:
import treelite

# Load the Treelite model (checkpoint file has been copied over)
checkpoint_path = "./checkpoint.tl"
tl_model = treelite.Model.deserialize(checkpoint_path)

# Make predictions using Treelite
out_prob = treelite.gtil.predict(tl_model, X, pred_margin=True)
print(out_prob)